# QPC

Defined a quantum point contact. 

We will here define two gates and one zone underaneath it with a charge density (2DEG GAS). 
We will then update the system for a number of charge densities and voltages using
the LinearProblem.update() method. 

## Geometry

### Shapes class

When defining the geometry in the previous examples we used poisson.continuous.shapes.Rectangle and Ellipsoid. They are both subclasses of General wich is itself a subclass of Shape (an ABC class). In oder to create a subclass of General one must only create a geometry(self, kwargs) module within the class that takes as parameter a numpy array and returns a numpy array of booleans. More info on that can be found on poisson.continuous.shapes. and an example of a subclass of Geometry is poisson.continuous.shapes.Rectangle. 

Here we are not going to use a subclass of General in order to construct our geometry. In fact we are going to use a subclass of InHull (an ABC class) which is itself a subclass of Shape. InHull takes a list of points ( vertices of your stucture, which must be convex) and performs a Deulanay tessalation, which allows us to define a function detecting if a set of coordinates is within the tessalation or not. Just as for the General class, one can create a subclass of InHull. In order to do so one only needs to creat a module within the afformanted class called generate_points(self) that returns an array of coordinates corresponding to the vertices of convex geometrical object. For an example of the latter one can check the class poisson.continuous.shapes.RegularPolygon. 

The advantage of creating using an InHull subclass, other than the logical operations available for all Shape subclasses, is the possibility to translate and rotate your geometrical object. This will be shown bellow. 

The class used here is Delaunay. This subclass of InHull takes as parameters the vertex of the convex geometrical object as parameter or an instance of scipy.spatial.qhull.Deulaunay. 

In [ ]:
import copy
import numpy as np
from poisson.continuous import shapes
import poisson

def make_QPC(alphab = 1, alphag = 0.25, alphagas = 0.25):
    '''
        Parameters:
        -----------
            alphab: step in the regions ouside of the 2D gas or electrodes. 
            alphagas: step on the zone defining the 2D gas
            alphag: step on the zone defining the electrodes
    '''
    ### gates
    gatez = 5  # lower position of the gate in z (on top of gas)
    gateh = 1  # heigth of the gates
    gateextw = 15  # width of the gate at side opposed to gas
    gateintw = 5  # width of the gate at side facing the gas
    gatedepth = 6   # x extension of the gate
    gatexpos = 3   # x position of the interior of the gate

    ### gas
    gasz = 0  # gas position in z
    gasx = 10   # gas extension along x
    gasy = 30  # gas extension along y

    ### refinement
    alphab = 1
    alphag = 0.25
    alphagas = 0.25

    ### external box
    extbox = np.array(
            [-(2 * gatexpos + gatedepth), (2 * gatexpos + gatedepth),
             -.75 * gasy, .75 * gasy, -10 * alphag, 2 * (gateh + gatez)])

    ### gates
    gatel = shapes.Delaunay(
            [[-(gatexpos + gatedepth) , -gateextw/2, gatez],
              [-(gatexpos + gatedepth) , gateextw/2, gatez],
              [-(gatexpos) , -gateintw/2, gatez],
              [-(gatexpos) , gateintw/2, gatez],
              [-(gatexpos + gatedepth) , -gateextw/2, gatez + gateh],
              [-(gatexpos + gatedepth) , gateextw/2, gatez + gateh],
              [-(gatexpos) , -gateintw/2, gatez + gateh],
              [-(gatexpos) , gateintw/2, gatez + gateh]])
    gatel_mesh = [extbox, alphag, gatel, 2]

    gater = copy.deepcopy(gatel)
    gater.rotate([0, 0, np.pi], center=[0, 0, 0])
    gater_mesh = [extbox, alphag, gater, 3]

    ### gas
    gas = shapes.Delaunay(
            [[-gasx/2, -gasy/2, gasz + 4*alphag],
             [gasx/2, -gasy/2, gasz + 4*alphag],
             [-gasx/2, gasy/2, gasz + 4*alphag],
             [gasx/2, gasy/2, gasz + 4*alphag],
             [-gasx/2, -gasy/2, gasz - 4*alphag],
             [gasx/2, -gasy/2, gasz - 4*alphag],
             [-gasx/2, gasy/2, gasz - 4*alphag],
             [gasx/2, gasy/2, gasz - 4*alphag]])
    gas_mesh = [extbox, alphagas, gas, 4]

    ### exterior box
    extbbox = shapes.Delaunay(
            [extbox[[0, 2, 4]], extbox[[0, 2, 5]],
             extbox[[0, 3, 4]], extbox[[0, 3, 5]],
             extbox[[1, 2, 4]], extbox[[1, 2, 5]],
             extbox[[1, 3, 4]], extbox[[1, 3, 5]]])
    ext_mesh = [extbox, alphab, extbbox, 1]

    # Build the grid object. 
    grid = poisson.GridBuilder(meshs=[ext_mesh, gatel_mesh, gater_mesh, gas_mesh])
    
    ### mesh
    return (grid, gatel, gater, gas, extbbox)


In [ ]:
# Visualize the grid. 
alphab = 1
alphag = 0.25
alphagas = 0.25
grid, t, t, t, t = make_QPC(alphab, alphag, alphagas)
poisson.plot.points_3D_mavi(grid, scale_factor=0.05)

### Define the Continuous Geometry

In [ ]:
grid, gatel, gater, gas, extbbox = make_QPC()

dielectric = (extbbox - (gatel + gater + gas))

# we can set the default dielectric to 12 * 8.85e-12 and have a constant dielectric constant
# everywhere by adding the parameter default_relative_permittivity = 12
# We can also set the dielectric constant at individual regions using the dielecetric parameter, 
# e.g. dielectric = [(gas, 10), (dielectric, 16), (gater + gatel, 3)]
geo_inst = poisson.ContinuousGeometry(
    voltage = {'gater':gater, 'gatel':gatel}, 
    charge = {'2DEG': gas, 'dielectric': dielectric},
    default_relative_permittivity = 12) 

## Discrete Poisson

In [ ]:
dis_poisson = poisson.DiscretePoisson(
    geo_inst, grid=grid,
    selection={'Neuman-Dirichlet':[['voltage', '*']]})

## Linear Problem

Here we need to check how we will transform a 2D density ($6. 10^{10} \text{cm}^{-2}$) into a value that can be sent to the poisson solver. 

The first thing is to transform the units from cm to nm -> $6. 10^{-4} \text{nm}^{-2}$

The thickness of our 2D gas is 4 times the step of the mesh on the gas. We can consider the density invariant along
that direction and transform the 2D density into a 3D density by dividing the 2D density by the thickness of the 2D gas, i.e. $\frac{6}{4 * \text{alphagas}}. 10^{-4} \text{nm}^{-3}$

In [ ]:
linear_prob = poisson.LinearProblem(dis_poisson,
                                    voltage_val=[(gatel, 5), (gater, 5)],
                                    charge_val=[(gas,  6e-4 / (4 * alphagas))],
                                    is_charge_density=True)

In [ ]:
from matplotlib import pyplot as plt

def plot(linear_problem_inst = None):

    linear_problem_inst.plot_3d('both', ('Voltage (V)', 'Charge'),
                        scale_factor=(0.05, 10))
    
    plot_voltage, plot_charge = linear_problem_inst.plot_cut_1d(
            directions=(1, 2), bbox='default', npoints=2000)

    fig2 = plt.figure()
    ax_voltage = fig2.add_subplot(111)

    t, data_volt = plot_voltage(
            (0, 0), ax = ax_voltage, marker='.', color='k',
            label='Voltage simulation', linestyle='None')

    ax_voltage.set_xlabel('{0}(nm)'.format('y'))
    ax_voltage.set_ylabel('Voltage(V)')
    ax_voltage.legend(loc='upper center')

    ax_charge = ax_voltage.twinx()

    tt, data_charge = plot_charge(
            (0, 0), ax = ax_charge, marker='.', color='b',
            label='Charge simulation', linestyle='None')

    ax_charge.set_xlabel('{0}(nm)'.format('y'))
    ax_charge.set_ylabel(r'Charge density $(\#.nm^{{{-2}}})$')
    ax_charge.legend(loc='lower center')

    plt.show()

    plot_voltage_2d, plot_charge_2d = linear_problem_inst.plot_cut_2d(direction=2)

    plot_voltage_2d(0, colorbar_label=r'Voltage (V)')
    plot_charge_2d(0, colorbar_label=r'Charge density($\#.nm^{{{-2}}}$)')

    plot_voltage_2d(5, colorbar_label=r'Voltage (V)')
    plot_charge_2d(5, colorbar_label=r'Charge density($\#.nm^{{{-2}}}$)')

    plt.show()  


In [ ]:
plot(linear_prob)

## Updating the voltage or charge density 

It is possible now to update the voltage and charge values without having to make a new capacitance matrix nor
to refactorize again the system of equations defined in LinearProblem. 

This is achieved with the LinearProblem.update and LinearProblem.solve modules. 

LinearProblem.update take as parameters voltage_val, charge_val and mixed_val ( explained in more detail in the next tutorial). They have the same format as the ones given to LinearProblem. 

LinearProblem.solve take as parameters the solver_type (default is MUMPS) which selects wich solver
will be used, new_instance (default False) which determines if a new instance of the Solver class will be 
created. Any other parameter is directly passed to the solver. 

In [ ]:
linear_prob.update(voltage_val=[(gatel, -5), (gater, -5)])
linear_prob.solve(solver_type='mumps', new_instance=False, factorize=False)

plot(linear_prob)

In [ ]:
linear_prob.update(charge_val=[(gas,  -6e-4 / (4 * alphagas))])
linear_prob.solve(solver_type='mumps', new_instance=False, factorize=False)

plot(linear_prob)